In [1]:
# !pip install -U gensim

In [2]:
from gensim.models import Word2Vec, KeyedVectors

In [3]:
import numpy as np

In [4]:
modelo = KeyedVectors.load_word2vec_format("../../../dados/skip_s100/skip_s100.txt")

In [5]:
modelo["gato"]

array([-0.164461, -0.014363,  0.124512, -0.303955,  0.160792, -0.032028,
       -0.048507, -0.034208,  0.107088, -0.093499,  0.349632, -0.087973,
       -0.101196,  0.054815,  0.237671, -0.061357,  0.264327,  0.298272,
       -0.024176,  0.389583,  0.065943,  0.054307,  0.106118,  0.23514 ,
        0.078463,  0.282537, -0.150036, -0.006788,  0.051575,  0.083578,
        0.242771, -0.223759, -0.146031, -0.159297,  0.287694, -0.083225,
       -0.158965, -0.046706, -0.039911,  0.224569,  0.168114,  0.083644,
        0.137285,  0.020159,  0.088486, -0.141225,  0.265956,  0.023687,
        0.166814,  0.052159, -0.223885, -0.215677,  0.429719,  0.225931,
        0.125467,  0.116944, -0.012855, -0.188196, -0.008294,  0.582053,
        0.032072,  0.036928,  0.016363, -0.003704, -0.225326,  0.073739,
        0.432913,  0.521782, -0.073163,  0.16542 ,  0.003852,  0.422371,
        0.14309 , -0.079214,  0.112097, -0.142931,  0.104638,  0.324669,
       -0.213694, -0.496184,  0.051481,  0.218193, 

In [6]:
gato_hipotenusa = np.sqrt(np.sum(modelo["gato"] ** 2))

In [7]:
gato_normal = modelo["gato"] / gato_hipotenusa

In [8]:
cahorro_hipotenusa = np.sqrt(np.sum(modelo["cahorro"] ** 2))

In [9]:
cachorro_normal = modelo["cachorro"] / cahorro_hipotenusa

In [10]:
np.dot(modelo["gato"], modelo["cachorro"]) / (gato_hipotenusa * cahorro_hipotenusa)

1.5357524

In [11]:
modelo.most_similar("transito")

[('trânsito', 0.877697229385376),
 ('tráfego', 0.6940882802009583),
 ('trafego', 0.6474871635437012),
 ('transporte', 0.642004668712616),
 ('ônibus.o', 0.6380943655967712),
 ('congestionado', 0.625903844833374),
 ('rodovário', 0.6216405630111694),
 ('manaustrans', 0.620560884475708),
 ('detran', 0.6070328950881958),
 ('rodoviário', 0.6058222651481628)]

In [12]:
modelo.most_similar(positive=["fêmea", "cachorro"], negative=["macho"])

[('cadela', 0.7303145527839661),
 ('bala', 0.6700485944747925),
 ('onça', 0.6677137017250061),
 ('gambá', 0.6570473313331604),
 ('cadelinha', 0.6529739499092102),
 ('garota', 0.6457968354225159),
 ('coleira', 0.6434794664382935),
 ('rêmora', 0.6320232152938843),
 ('égua', 0.6318328976631165),
 ('pulguinha', 0.6285112500190735)]

In [13]:
modelo.most_similar(positive=["mulher", "rei"], negative=["homem"])

[('rainha-consorte', 0.7912216186523438),
 ('primogénita', 0.7738461494445801),
 ('imperatriz-mãe', 0.7646884322166443),
 ('paleóloga', 0.752788245677948),
 ('dama-de-companhia', 0.7478024363517761),
 ('consorte', 0.7475903630256653),
 ('princesa-eleitora', 0.7472771406173706),
 ('piroska', 0.7468665838241577),
 ('ulrica', 0.7454056143760681),
 ('ranavalona', 0.7441917657852173)]

In [14]:
modelo.doesnt_match(["gato", "cachorro", "papagaio", "casa", "galinha"])

'casa'

In [15]:
modelo.doesnt_match(["dinheiro", "estudo", "dormir", "trabalhar", "estudante"])

'estudo'

# Analise de Sentimento utilizando Word Embedding

In [16]:
import pandas as pd

In [17]:
import matplotlib.pyplot as plt

In [18]:
import nltk

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
from tensorflow.keras.models import Sequential

In [21]:
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D

In [22]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [23]:
import string

In [24]:
from unidecode import unidecode

In [25]:
from sklearn.preprocessing import LabelEncoder

In [26]:
from tensorflow.keras.utils import to_categorical

In [27]:
df = pd.read_csv("../../../dados/financial_phrase/financial_phrase_bank_pt_br.csv", low_memory=False)

In [28]:
df = df[["y", "text_pt"]]

In [29]:
df

,y,text_pt
0,neutral,A Technopolis planeja desenvolver em etapas um...
1,negative,"A Elcoteq, empresa internacional da indústria ..."
2,positive,Com a nova planta de produção a empresa aument...
3,positive,De acordo com a estratégia atualizada da empre...
4,positive,FINANCIAMENTO DO CRESCIMENTO DA ASPOCOMP A Asp...
...,...,...
4840,negative,LONDRES MarketWatch - Os preços das ações term...
4841,neutral,"As vendas de cerveja da Rinkuskiai caíram 6,5 ..."
4842,negative,"O lucro operacional caiu para EUR 35,4 milhões..."
4843,negative,As vendas líquidas do segmento de Papel diminu...


In [30]:
# preparar o texto removendo caracteres especiais e pontuação
def preparar_texto( texto ):
    texto_minusculo = texto.lower()
    mascara = str.maketrans("\n\r\t", "   ", string.punctuation)
    texto_limpo = texto_minusculo.translate(mascara)
    texto_limpo = unidecode(texto_limpo)
    return texto_limpo

In [31]:
count = 0
def limpar_texto( texto ):
    global count
    texto_limpo = ""
    if type(texto) == str:
        texto_limpo = preparar_texto( texto )
    count += 1
    if count % 2000 == 0:
        print(f"Preparando texto: {count}")
    return texto_limpo

df['clean_text'] = df["text_pt"].apply(limpar_texto)

Preparando texto: 2000
Preparando texto: 4000


In [32]:
unique_classes = df['y'].unique()

In [33]:
NUM_CLASSES = len(unique_classes)

In [34]:
# for word in modelo.index_to_key:
#    print(word)

In [35]:
# modelo["</s>"]

In [36]:
df["clean_text"]

0       a technopolis planeja desenvolver em etapas um...
1       a elcoteq empresa internacional da industria e...
2       com a nova planta de producao a empresa aument...
3       de acordo com a estrategia atualizada da empre...
4       financiamento do crescimento da aspocomp a asp...
                              ...                        
4840    londres marketwatch  os precos das acoes termi...
4841    as vendas de cerveja da rinkuskiai cairam 65 p...
4842    o lucro operacional caiu para eur 354 milhoes ...
4843    as vendas liquidas do segmento de papel diminu...
4844    as vendas na finlandia diminuiram 105 em janei...
Name: clean_text, Length: 4845, dtype: object

In [37]:
max_palavras = 0
def texto_to_embedding( texto ):
    global max_palavras
    texto_embedding = []
    for indice_palavra, palavra in enumerate(texto.split(" ")):
        try:
            vetor = modelo[palavra]
        except:
            vetor = modelo['</s>'] # np.zeros(100)
            # palavra desconhecida
        texto_embedding.append(np.array(vetor))
        if indice_palavra > max_palavras:
            max_palavras = indice_palavra
    return np.array(texto_embedding)

In [38]:
texto_to_embedding(df['clean_text'][2]).shape

(31, 100)

In [39]:
count = 0
def gerar_embeddings( texto ):
    global count
    embedding = texto_to_embedding(texto)
    count += 1
    if count % 500 == 0:
        print(f"Preparando texto: {count}")
    return embedding


In [40]:
df['embedding_text'] = df["clean_text"].apply(gerar_embeddings)

Preparando texto: 500
Preparando texto: 1000
Preparando texto: 1500
Preparando texto: 2000
Preparando texto: 2500
Preparando texto: 3000
Preparando texto: 3500
Preparando texto: 4000
Preparando texto: 4500


In [41]:
embedding_text = df["embedding_text"][2]
lista = [elemento for elemento in embedding_text]
if len(embedding_text) < max_palavras:
    amais = max_palavras - len(embedding_text)
    for i in range(amais):
        lista.append(np.zeros(100))
embedding_padded_lista = np.array(lista)
embedding_padded_lista.shape

(63, 100)

In [42]:
def pad_embedding_texts( texto_embedding ):
    global max_palavras
    lista = [elemento for elemento in embedding_text]
    if len(embedding_text) < max_palavras:
        amais = max_palavras - len(embedding_text)
        for i in range(amais):
            lista.append(np.zeros(100))
    embedding_padded_lista = np.array(lista)
    return embedding_padded_lista

In [43]:
df['embedding_padded_text'] = df["embedding_text"].apply(pad_embedding_texts)

In [44]:
df['embedding_padded_text'][1000].shape

(63, 100)

In [45]:
# X = np.array(df['embedding_padded_text'])
# X

In [46]:
label_encoder = LabelEncoder()

In [47]:
Y_numeros = label_encoder.fit_transform(df['y'])

In [48]:
Y = to_categorical(Y_numeros)

In [49]:
X = np.array([np.array(embed_item) for embed_item in df['embedding_padded_text']])

In [50]:
X.shape

(4845, 63, 100)

In [51]:
model = Sequential()
model.add(Dense(128, input_shape=(max_palavras, 100), activation="relu"))
model.add(GlobalAveragePooling1D())
# model.add(Dense(64, activation="relu"))
model.add(Dense(3, activation="softmax"))
model.summary()

C:\Users\anton\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 63, 128)             │          12,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ (None, 128)                 │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │             387 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 13,315 (52.01 KB)

 Trainable params: 13,315 (52.01 KB)

 Non-trainable params: 0 (0.00 B)

In [52]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [53]:
model.fit(X, Y, epochs=10)

Epoch 1/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.5865 - loss: 0.9706
Epoch 2/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5992 - loss: 0.9121
Epoch 3/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5966 - loss: 0.9240
Epoch 4/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.5775 - loss: 0.9385
Epoch 5/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5974 - loss: 0.9212
Epoch 6/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5878 - loss: 0.9294
Epoch 7/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5947 - loss: 0.9230
Epoch 8/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5925 - loss: 0.9317
Epoch 9/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6109 - loss: 0.9080
Epoch 10/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5978 - loss: 0.9203
